 **AAB (Android App Bundle)** in React Native for Play Store, follow these steps:

---

## ✅ Step-by-Step: Generate and Release `.aab` File

---

### 🔧 1. **Create a Keystore (Only Once)**

In terminal:

```bash
keytool -genkey -v -keystore my-release-key.keystore -alias my-key-alias -keyalg RSA -keysize 2048 -validity 10000
```

It will ask for passwords and information. Save them securely.

> This generates `my-release-key.keystore`.

Move it to:

```
android/app/my-release-key.keystore
```

---

### ✍️ 2. **Configure `gradle.properties`**

Edit this file:

```
android/gradle.properties
```

Add:

```properties
MYAPP_UPLOAD_STORE_FILE=my-release-key.keystore
MYAPP_UPLOAD_KEY_ALIAS=my-key-alias
MYAPP_UPLOAD_STORE_PASSWORD=your-store-password
MYAPP_UPLOAD_KEY_PASSWORD=your-key-password
```

---

### 🛠️ 3. **Update `android/app/build.gradle`**

Find the `signingConfigs` block and update it:

```groovy
signingConfigs {
    release {
        storeFile file(MYAPP_UPLOAD_STORE_FILE)
        storePassword MYAPP_UPLOAD_STORE_PASSWORD
        keyAlias MYAPP_UPLOAD_KEY_ALIAS
        keyPassword MYAPP_UPLOAD_KEY_PASSWORD
    }
}
```

Then update the `buildTypes` section:

```groovy
buildTypes {
    release {
        signingConfig signingConfigs.release
        shrinkResources true
        minifyEnabled true
        proguardFiles getDefaultProguardFile("proguard-android-optimize.txt"), "proguard-rules.pro"
    }
}
```

---

### 🧱 4. **Build the `.aab` File**

In your terminal:

```bash
cd android
./gradlew bundleRelease
```

> Output file:

```
android/app/build/outputs/bundle/release/app-release.aab
```

---

### 🚀 5. **Upload `.aab` to Google Play Console**

1. Go to [https://play.google.com/console](https://play.google.com/console)
2. Create or select an app.
3. Go to **"Release > Production"** or **"Internal Testing"**
4. Upload `app-release.aab`
5. Fill in the release info and submit for review.

---

## 🔒 Security Tip

Avoid committing `keystore` and passwords into Git!

You can use `.env` or GitHub Secrets for CI/CD automation later.

---

Let me know if you need help with:

* Testing with internal users
* Creating `.env` file to keep keystore credentials secret
* Setting up CI/CD with GitHub Actions or Bitrise



# 2

 a **simple Bash script** to automate the process of building a signed `.aab` (Android App Bundle) file in a React Native project.

---

## ✅ `build-aab.sh` – Script to Build `.aab` Automatically

### 📁 Create the Script File

1. In your project root, create a file:

```bash
touch build-aab.sh
chmod +x build-aab.sh
```

2. Paste the following content into `build-aab.sh`:

```bash
#!/bin/bash

echo "🔧 Starting Android App Bundle (.aab) build..."

# Step 1: Clean previous builds
cd android || exit
echo "🧹 Cleaning old builds..."
./gradlew clean

# Step 2: Build .aab file
echo "📦 Building the release .aab..."
./gradlew bundleRelease

# Step 3: Output path
AAB_PATH="app/build/outputs/bundle/release/app-release.aab"

# Step 4: Check if file was generated
if [ -f "$AAB_PATH" ]; then
    echo "✅ .aab build successful!"
    echo "📁 Output: android/$AAB_PATH"
else
    echo "❌ Build failed. Check error logs above."
fi
```

---

## 🧪 Run the Script

From your project root:

```bash
./build-aab.sh
```

---

## 💡 Optional: Rename or Copy `.aab` to Output Folder

To automatically move or rename the AAB file after build, add this to the script before the final `if`:

```bash
mkdir -p ../../output
cp $AAB_PATH ../../output/MyApp-$(date +%Y%m%d-%H%M).aab
```

---

### 🔐 Reminder:

Ensure your `gradle.properties` is configured with your signing details as explained earlier.

---

Let me know if you want a **Windows `.bat` version** too or need this script integrated into a **CI/CD workflow** like GitHub Actions!


Awesome! Here's everything you asked for:

---

## ✅ 1. **Windows `.bat` Script** to Build Signed `.aab`

Create a file in your project root named: `build-aab.bat`

### 🪟 `build-aab.bat` (for Windows CMD)

```bat
@echo off
echo 🔧 Starting Android App Bundle (.aab) build...

REM Step 1: Navigate to android directory
cd android

REM Step 2: Clean old builds
echo 🧹 Cleaning old builds...
gradlew clean

REM Step 3: Build .aab
echo 📦 Building the release .aab...
gradlew bundleRelease

REM Step 4: Check if .aab exists
set AAB_PATH=app\build\outputs\bundle\release\app-release.aab

if exist %AAB_PATH% (
    echo ✅ .aab build successful!
    echo 📁 Output: android\%AAB_PATH%
) else (
    echo ❌ Build failed. Check logs above.
)
```

> ⚠️ You can run it by double-clicking or from terminal:

```bash
build-aab.bat
```

---

## ✅ 2. **GitHub Actions Workflow** to Build `.aab`

### 📁 Folder: `.github/workflows/android-aab.yml`

Create this file inside your repo:

```yaml
name: Build Android AAB

on:
  workflow_dispatch:  # Manual trigger

jobs:
  build:
    runs-on: ubuntu-latest

    steps:
      - name: 📥 Checkout code
        uses: actions/checkout@v3

      - name: 🟢 Set up JDK
        uses: actions/setup-java@v3
        with:
          distribution: 'temurin'
          java-version: 17

      - name: 🧩 Install dependencies
        run: |
          cd android
          ./gradlew dependencies

      - name: 🔐 Setup signing credentials
        run: |
          echo "${{ secrets.KEYSTORE_BASE64 }}" | base64 -d > android/app/my-release-key.keystore
          echo "MYAPP_UPLOAD_STORE_FILE=my-release-key.keystore" >> android/gradle.properties
          echo "MYAPP_UPLOAD_KEY_ALIAS=${{ secrets.KEY_ALIAS }}" >> android/gradle.properties
          echo "MYAPP_UPLOAD_STORE_PASSWORD=${{ secrets.KEYSTORE_PASSWORD }}" >> android/gradle.properties
          echo "MYAPP_UPLOAD_KEY_PASSWORD=${{ secrets.KEY_PASSWORD }}" >> android/gradle.properties

      - name: 📦 Build .aab
        run: |
          cd android
          ./gradlew bundleRelease

      - name: 📤 Upload AAB Artifact
        uses: actions/upload-artifact@v3
        with:
          name: app-release.aab
          path: android/app/build/outputs/bundle/release/app-release.aab
```

---

### 🔐 Secrets to Add in GitHub

Go to **GitHub repo → Settings → Secrets and variables → Actions → New repository secret** and add:

| Secret Name         | Value                        |
| ------------------- | ---------------------------- |
| `KEYSTORE_BASE64`   | Base64-encoded keystore file |
| `KEY_ALIAS`         | Alias used in keystore       |
| `KEYSTORE_PASSWORD` | Keystore password            |
| `KEY_PASSWORD`      | Key password                 |

To get the `KEYSTORE_BASE64`, run:

```bash
base64 android/app/my-release-key.keystore > keystore.txt
```

Then copy the contents of `keystore.txt` and paste it as the `KEYSTORE_BASE64` secret.

---

### ✅ Result:

* Trigger this workflow manually from GitHub Actions tab.
* It builds the `.aab` and uploads it as a downloadable artifact.

---

Would you also like:

* **CI deployment to Play Store (via Fastlane)?**
* Or **internal testing track upload**?

Let me know and I’ll guide you through that too.
